# Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import optimizationFuncs as optim
import cpso
Adaswarm = cpso.Adaswarm

## Reload

In [ ]:
from importlib import reload

cpso = reload(cpso)
Adaswarm = cpso.Adaswarm

## Initialise Swarm

In [ ]:
# Number of particles
Np = 25

# Dimension of the problem
dims = 1

# Left boundary --> 5 dimensions, and x_i=-100 is the left boundary each dimension (i=0,1,2,3,4)
llim = np.repeat (-20, dims)

# Right boundary --> 5 dimensions, and x_i=100 is the right boundary for each dimension (i=0,1,2,3,4)
rlim = np.repeat (20, dims)

# Returns the swarm --> Also internally creates a chaotic generator object
swarm = Adaswarm.get_chaotic_swarm(
    {
        'init_cmap'    : {
            'name'      : 'logistic',        # Initialisation map --> Logistic map
            'args'      : ()                 # Default logistic parameter --> 4
        },
        
        'dyn_cmap'     : {
            'name'      : 'tent',            # Chaotic map for r1, r2 --> Tent map
            'args'      : ()                 # Default tent paraneter --> 0.49999
        }
    }
)

## Supply Objective Function

In [ ]:
objective = optim.rastrigin

# This call creates the PSO optimizer object
mizer = swarm(objective, llim, rlim, Np)

## Optimize

In [ ]:
# Returns (optimum, approx-grad-function)
retpack = mizer.optimize()
opt, approxGrad = retpack['rets']
print ("\rOptimum is at f({}) = {}".format(opt, objective(opt.reshape(1,-1))[0]))
print ("Gradient at optimum = {}".format(approxGrad(opt)[0]))

## Check Grad

In [ ]:
# Perturbing the optimum
dopt = opt + 1e-5*np.random.random_sample (opt.shape)
print ("approxGrad({}) = {}".format(dopt, approxGrad(dopt).flatten()))

## Plot of 1-D approx gradient vs. true gradient

In [ ]:
assert llim.shape[0] == 1

truGrad = optim.funcGrads[objective]
x = (lambda dx:np.linspace(opt-dx, opt+dx, 1000))(min(opt[0]-llim[0], rlim[0]-opt[0])/10)
print ("Objective function info --> \n{}".format(objective.__doc__))

fig, ax = plt.subplots(1,2, figsize=(15,7.5))
ax[0].plot(opt, truGrad(opt.reshape(1,-1)).flatten(), '*r', label="Opt")
ax[0].plot(x, truGrad(x), '-b', label="True grad")
ax[0].plot(x, -approxGrad(x), '--g', label="Approx grad")
ax[1].plot(opt, objective(opt.reshape(1,-1)).flatten(), '*r', label="Opt")
ax[1].plot(x, objective(x), 'b', label="Objective")
leg = ax[0].legend();